# **Example 1**

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston

import pandas as pd

import plotly.express as px
import plotly
import plotly.graph_objs as go

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), 'Physical GPUs,', len(logical_gpus), 'Logical GPUs')
    except RuntimeError as e:
        print(e)

In [ ]:
x, y = make_regression(n_samples=3000, n_features=10, noise=0.2, random_state=1)
x.shape, y.shape

In [ ]:
x = StandardScaler().fit_transform(x)
y = StandardScaler().fit_transform(y.reshape(len(y),1))[:,0]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.5, shuffle= True)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
x_embedded = TSNE(n_components=1, random_state=99, verbose=1, perplexity=40, n_iter=5000).fit_transform(x_train)
x_embedded.shape

In [ ]:
x_train_pd = pd.DataFrame(x_embedded, columns=['x'])
y_train_pd = pd.DataFrame(y_train, columns=['y'])

In [ ]:
df = pd.concat([x_train_pd, y_train_pd], axis=1)

In [ ]:
fig = px.scatter(df, x='x', y='y')
fig.show()

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(20, input_dim=10, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

**Compile Model โดยกำหนด Loss Function เป็น Mean Squared Error**

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='mean_squared_error', optimizer=opt)

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, verbose=1)

In [ ]:
h1 = go.Scatter(y=history.history['loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='blue'),
                        name="loss"
                   )
h2 = go.Scatter(y=history.history['val_loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='red'),
                        name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1)

In [ ]:
train_mse = model.evaluate(x_train, y_train, verbose=0)
val_mse = model.evaluate(x_val, y_val, verbose=0)
print('Train: %.5f, Validation: %.5f' % (train_mse, val_mse))

# **Example 2 : Mean Squared Logarithmic Error Loss**

In [ ]:
boston_dataset = load_boston()
boston_dataset.keys()

In [ ]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

In [ ]:
boston['PRICE'] = boston_dataset.target
boston.head()

In [ ]:
sns.set(rc={'figure.figsize':(11,8)})
sns.histplot(boston['PRICE'], bins=30, kde=True)
plt.show()

In [ ]:
correlation_matrix = boston.corr().round(3)

sns.heatmap(data=correlation_matrix, annot=True)
plt.savefig('correlation_matrix.jpeg', dpi=300)

In [ ]:
plt.figure(figsize=(25, 10))

features = ['RM', 'LSTAT']
target = boston['PRICE']

for i, col in enumerate(features):
    plt.subplot(1, len(features) , i+1)
    x = boston[col]
    y = target
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('PRICE')

plt.savefig('scatter.jpeg', dpi=300)

In [ ]:
x = pd.DataFrame(np.c_[boston['LSTAT'], boston['RM']], columns = ['LSTAT','RM'])
y = boston['PRICE']

x.shape, y.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state=5)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
model.compile(loss='mean_squared_error', optimizer=opt)

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, verbose=1)

In [ ]:
h1 = go.Scatter(y=history.history['loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='blue'),
                        name="loss"
                   )
h2 = go.Scatter(y=history.history['val_loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='red'),
                        name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1)

In [ ]:
train_mse = model.evaluate(x_train, y_train, verbose=0)
val_mse = model.evaluate(x_val, y_val, verbose=0)
print('Train: %.5f, Validation: %.5f' % (train_mse, val_mse))

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
model.compile(loss='mean_squared_logarithmic_error', optimizer=opt, metrics=['mse'])

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, verbose=1)

In [ ]:
h1 = go.Scatter(y=history.history['loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='blue'),
                        name="loss"
                   )
h2 = go.Scatter(y=history.history['val_loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='red'),
                        name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1)

In [ ]:
train_msle = model.evaluate(x_train, y_train, verbose=0)
val_msle = model.evaluate(x_val, y_val, verbose=0)
print('MSLE Train: %.5f, Validation: %.5f' % (train_msle[0], val_msle[0]))
print('MSE Train: %.5f, Validation: %.5f' % (train_msle[1], val_msle[1]))

## **Example 3**

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
for i in range(9):
    plt.subplot(3, 3, 1 + i)    
    img = train_images[i].reshape(train_images[0].shape[0], train_images[0].shape[1])
    plt.axis('off')
    plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:
noise_factor = 0.5
x_train_noisy = train_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_images.shape) 
x_test_noisy = test_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_images.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
for i in range(9):
    plt.subplot(3, 3, 1 + i)    
    
    img = x_train_noisy[i].reshape(x_train_noisy[0].shape[0], x_train_noisy[0].shape[1])
    plt.axis('off')
    plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:
#Encoder
inp = tf.keras.layers.Input((28, 28,1))
e = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inp)
e = tf.keras.layers.BatchNormalization()(e)

e = tf.keras.layers.MaxPooling2D((2, 2))(e)
e = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(e)
e = tf.keras.layers.BatchNormalization()(e)

e = tf.keras.layers.MaxPooling2D((2, 2))(e)
e = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(e)
e = tf.keras.layers.BatchNormalization()(e)

l = tf.keras.layers.Flatten()(e)
l = tf.keras.layers.Dense(49, activation='relu')(l)

In [ ]:
#DECODER
d = tf.keras.layers.Reshape((7,7,1))(l)
d = tf.keras.layers.Conv2DTranspose(64,(3, 3), strides=2, activation='relu', padding='same')(d)
d = tf.keras.layers.BatchNormalization()(d)
d = tf.keras.layers.Conv2DTranspose(64,(3, 3), strides=2, activation='relu', padding='same')(d)
d = tf.keras.layers.BatchNormalization()(d)
d = tf.keras.layers.Conv2DTranspose(32,(3, 3), activation='relu', padding='same')(d)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(d)
ae = tf.keras.Model(inp, decoded)

ae.summary()

In [ ]:
ae.compile(optimizer='adam', loss='mse')

In [ ]:
ae.fit(x_train_noisy, train_images, epochs=2, verbose=1)

In [ ]:
test_prediction = ae.predict(x_test_noisy, verbose=1, batch_size=100)

In [ ]:
for i in range(9):
    plt.subplot(3, 3, 1 + i)    
    
    img = x_test_noisy[i].reshape(x_test_noisy[0].shape[0], x_test_noisy[0].shape[1])
    plt.axis('off')
    plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:
for i in range(9):
    plt.subplot(3, 3, 1 + i)    
    
    x = test_prediction[i].reshape(28,28)
    plt.axis('off')
    plt.imshow(x, cmap=plt.get_cmap('gray'))

In [ ]:
#Encoder
inp = tf.keras.layers.Input((28, 28,1))
e = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inp)
e = tf.keras.layers.BatchNormalization()(e)

e = tf.keras.layers.MaxPooling2D((2, 2))(e)
e = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(e)
e = tf.keras.layers.BatchNormalization()(e)

e = tf.keras.layers.MaxPooling2D((2, 2))(e)
e = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(e)
e = tf.keras.layers.BatchNormalization()(e)

l = tf.keras.layers.Flatten()(e)
l = tf.keras.layers.Dense(49, activation='relu')(l)

In [ ]:
#Decoder
d = tf.keras.layers.Reshape((7,7,1))(l)
d = tf.keras.layers.Conv2DTranspose(64,(3, 3), strides=2, activation='relu', padding='same')(d)
d = tf.keras.layers.BatchNormalization()(d)
d = tf.keras.layers.Conv2DTranspose(64,(3, 3), strides=2, activation='relu', padding='same')(d)
d = tf.keras.layers.BatchNormalization()(d)
d = tf.keras.layers.Conv2DTranspose(32,(3, 3), activation='relu', padding='same')(d)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(d)
ae = tf.keras.Model(inp, decoded)

In [ ]:
ae.compile(optimizer="adam", loss="mae")

In [ ]:
ae.fit(x_train_noisy, train_images, epochs=2, verbose=1)

In [ ]:
test_prediction = ae.predict(x_test_noisy, verbose=1, batch_size=100)

In [ ]:
for i in range(9):
    plt.subplot(3, 3, 1 + i)    
    
    x = test_prediction[i].reshape(28,28)
    plt.axis('off')
    plt.imshow(x, cmap=plt.get_cmap('gray'))